In [1]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from keras.preprocessing.image import ImageDataGenerator
# from skimage import io
# import tensorflow as tf
# from tensorflow.keras import Input, layers
# from tensorflow.keras import models
# from tensorflow.keras.layers.experimental import preprocessing
# from tensorflow.lite.experimental.microfrontend.python.ops import audio_microfrontend_op as frontend_op
# print(tf.__version__)
# import numpy as np
# import shutil, os
# import matplotlib.pyplot as plt
# import pathlib
# import random
# import sys
# import io
# import os
# import glob
# from datetime import datetime as dt


In [2]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [3]:
def train_gen_norm(dirName):
    train_datagen = ImageDataGenerator(
    #         samplewise_center=True,
    #         samplewise_std_normalization=True,   
             rescale=1./255,
    #         rotation_range=45,     #Random rotation between 0 and 45
    #         width_shift_range=0.1,   #% shift
    #         height_shift_range=0.1,
    #         shear_range=0.1,
    #         zoom_range=0.1,
    #         horizontal_flip=True,
    #         fill_mode='reflect'
            )    
        # this is a generator that will read pictures found in
    # subfolers of 'data/train', and indefinitely generate
    # batches of augmented image data
    train_generator = train_datagen.flow_from_directory(
            dirName,  # this is the input directory
            target_size=(IMG_SIZE, IMG_SIZE),  # all images will be resized to 64x64
            batch_size=BATCH_SIZE,
            color_mode="grayscale",
            class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels
    return train_generator

In [4]:
def train_gen_all(dirName):
    train_datagen = ImageDataGenerator(
            samplewise_center=True,
            samplewise_std_normalization=True,   
            rescale=1./255,
            rotation_range=45,     #Random rotation between 0 and 45
            width_shift_range=0.1,   #% shift
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='reflect'
            )    
        # this is a generator that will read pictures found in
    # subfolers of 'data/train', and indefinitely generate
    # batches of augmented image data
    train_generator = train_datagen.flow_from_directory(
            dirName,  # this is the input directory
            target_size=(IMG_SIZE, IMG_SIZE),  # all images will be resized to 64x64
            batch_size=BATCH_SIZE,
            color_mode="grayscale",
            class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels
    return train_generator

In [5]:
def train_gen(dirName):
    train_datagen = ImageDataGenerator(
    #         samplewise_center=True,
    #         samplewise_std_normalization=True,   
    #         rescale=1./255,
    #         rotation_range=45,     #Random rotation between 0 and 45
    #         width_shift_range=0.1,   #% shift
    #         height_shift_range=0.1,
    #         shear_range=0.1,
    #         zoom_range=0.1,
    #         horizontal_flip=True,
    #         fill_mode='reflect'
            )    
        # this is a generator that will read pictures found in
    # subfolers of 'data/train', and indefinitely generate
    # batches of augmented image data
    train_generator = train_datagen.flow_from_directory(
            dirName,  # this is the input directory
            target_size=(IMG_SIZE, IMG_SIZE),  # all images will be resized to 64x64
            batch_size=BATCH_SIZE,
            color_mode="grayscale"
    ) # since we use binary_crossentropy loss, we need binary labels
    return train_generator

In [6]:
def val_gen_norm(dirName):
        # this is the augmentation configuration we will use for training


    # this is the augmentation configuration we will use for validation:
    # only rescaling. But you can try other operations
    validation_datagen = ImageDataGenerator(
              rescale=1./255
             )

    #train_datagen.fit(unknown_filenames)




    # this is a similar generator, for validation data
    validation_generator = validation_datagen.flow_from_directory(
            dirName,
            target_size=(IMG_SIZE, IMG_SIZE),
            batch_size=BATCH_SIZE,
            color_mode="grayscale",
            class_mode='binary')
    return validation_generator

In [7]:
def val_gen(dirName):
    validation_datagen = ImageDataGenerator(
             )
    validation_generator = validation_datagen.flow_from_directory(
            dirName,
            target_size=(IMG_SIZE, IMG_SIZE),
            batch_size=BATCH_SIZE,
            color_mode="grayscale",
    )
    return validation_generator

In [8]:
def model_compile_mobile():
    model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )

In [9]:
def model_compile():
    model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
    )

In [10]:
def save_model():
    date_str = dt.now().strftime("%d%b%Y_%H%M").lower()
    model_file_name = f"kws_ufe_{date_str}.h5" 
    print(f"Saving model to {model_file_name}")
    model.save(model_file_name, overwrite=False)

In [11]:
def plot_accuracy(metric):
    metrics = metric.history
    plt.subplot(2,1,1)
    plt.semilogy(history.epoch, metrics['loss'], metrics['val_loss'])
    plt.legend(['training', 'validation'])
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.subplot(2,1,2)
    plt.plot(history.epoch, metrics['accuracy'], metrics['val_accuracy'])
    plt.legend(['training', 'validation'])
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.show()

In [12]:
def test_model(how_many=80000):
    data_dir = pathlib.Path(os.path.join('/Users/Jeromey/IOT/Project2/data/augmented/person'))
    bird_list = tf.io.gfile.glob(str(data_dir) + '/*')
    data_dir = pathlib.Path(os.path.join('/Users/Jeromey/IOT/Project2/data/augmented/notperson'))
    unknown_list = tf.io.gfile.glob(str(data_dir) + '/*')    
    
    random_unknown = list(range(0, len(bird_list))) #create array [1,2,3...]
    random.shuffle(random_unknown) #randomize the array
    
    bird=0
    for i in range(how_many):
        x1=tf.keras.utils.load_img(
            path=bird_list[random_unknown[i]],
            color_mode="grayscale",
            target_size=(96,96),
            interpolation='nearest'
            )
        x1_arr=tf.keras.utils.img_to_array(x1)
        x1_shape=x1_arr.reshape((1,) +x1_arr.shape)
        x1_predict=model.predict(x1_shape)
        prediction = np.argmax(x1_predict, axis=1)
        if (prediction==1):
            bird=bird+1
    #     if ((i%100)==0):
    #         print(f"{i}")
    print("Bird accuracy:", bird/how_many)

    unknown=0
    for i in range(how_many):
        x1=tf.keras.utils.load_img(
            path=unknown_list[random_unknown[i]],
            color_mode="grayscale",
            target_size=(96,96),
            interpolation='nearest'
            )
        x1_arr=tf.keras.utils.img_to_array(x1)
        x1_shape=x1_arr.reshape((1,) +x1_arr.shape)
        x1_predict=model.predict(x1_shape)
        prediction = np.argmax(x1_predict, axis=1)
        if (prediction==0):
            unknown=unknown+1
    #     if ((i%100)==0):
    #         print(f"{i}")
    print("Unknown accuracy:", unknown/how_many)